In [1]:
%tb
import gym
from tf2rl.algos.sac_discrete import SACDiscrete
from CustomTrainer import Trainer


parser = Trainer.get_argument()
parser.add_argument("-f")
parser = SACDiscrete.get_argument(parser)
parser.set_defaults(test_interval=2000)
parser.set_defaults(max_steps=6000)
parser.set_defaults(gpu=-1)
parser.set_defaults(n_warmup=500)
parser.set_defaults(batch_size=32)
parser.set_defaults(memory_capacity=int(6000))
args = parser.parse_args()

env = gym.make("CartPole-v0")
test_env = gym.make("CartPole-v0")
policy = SACDiscrete(
    state_shape=env.observation_space.shape,
    action_dim=env.action_space.n,
    discount=0.99,
    gpu=args.gpu,
    memory_capacity=args.memory_capacity,
    batch_size=args.batch_size,
    n_warmup=args.n_warmup)
trainer = Trainer(policy, env, args, test_env=test_env)
trainer()

No traceback available to show.
22:27:48.061 [INFO] (CustomTrainer.py:104) Total Epi:     1 Steps:      20 Episode Steps:    20 Return:  20.0000 FPS: 39482.85
22:27:48.063 [INFO] (CustomTrainer.py:104) Total Epi:     2 Steps:      34 Episode Steps:    14 Return:  14.0000 FPS: 47830.54
22:27:48.064 [INFO] (CustomTrainer.py:104) Total Epi:     3 Steps:      52 Episode Steps:    18 Return:  18.0000 FPS: 48568.71
22:27:48.065 [INFO] (CustomTrainer.py:104) Total Epi:     4 Steps:      65 Episode Steps:    13 Return:  13.0000 FPS: 44701.34
22:27:48.067 [INFO] (CustomTrainer.py:104) Total Epi:     5 Steps:      85 Episode Steps:    20 Return:  20.0000 FPS: 24766.64
22:27:48.070 [INFO] (CustomTrainer.py:104) Total Epi:     6 Steps:     124 Episode Steps:    39 Return:  39.0000 FPS: 33784.46
22:27:48.071 [INFO] (CustomTrainer.py:104) Total Epi:     7 Steps:     134 Episode Steps:    10 Return:  10.0000 FPS: 32619.08
22:27:48.073 [INFO] (CustomTrainer.py:104) Total Epi:     8 Steps:     150 Epis

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [6]:
import pickle
import numpy as np

In [2]:
pickle.dump(trainer.replay_buffer, open("./CARTPOLE_SAC_6000_RB.pkl", "wb"))


NameError: name 'trainer' is not defined

In [7]:
replay_buffer = pickle.load(open("./CARTPOLE_SAC_6000_RB.pkl", "rb"))

In [8]:
trb = replay_buffer.transform(replay_buffer.storage)
trb

defaultdict(list,
            {'obs': array([[ 0.03638951, -0.02160182,  0.01547802, -0.0296219 ],
                    [ 0.03595747,  0.1732948 ,  0.01488558, -0.31738144],
                    [ 0.03942337, -0.02203598,  0.00853795, -0.02004154],
                    ...,
                    [-0.05529382,  0.16934597, -0.01125058, -0.35236245],
                    [-0.0519069 , -0.02561419, -0.01829783, -0.06324829],
                    [-0.05241919, -0.22046909, -0.01956279,  0.22360586]],
                   dtype=float32),
             'act': array([[1.],
                    [0.],
                    [0.],
                    ...,
                    [0.],
                    [0.],
                    [1.]], dtype=float32),
             'next_obs': array([[ 0.03595747,  0.1732948 ,  0.01488558, -0.31738144],
                    [ 0.03942337, -0.02203598,  0.00853795, -0.02004154],
                    [ 0.03898265, -0.21727933,  0.00813712,  0.27532294],
                    ...,
      

In [9]:
trb = np.concatenate((trb["obs"], trb["act"], trb["next_obs"], trb["rew"], trb["done"]), axis=1)

In [10]:
trb.shape

(6000, 11)

In [11]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices(trb).shuffle(trb.shape[0]).batch(32)
test_dataset = train_dataset

In [12]:
from tensorflow.keras.layers import Dense, InputLayer
latent_dim = 8

encoder_layers = [64, 64, 128, 256]
decoder_layers = encoder_layers[::-1]

In [13]:
encoder = tf.keras.Sequential([
    InputLayer(input_shape=trb.shape[1:]),
    *[Dense(x) for x in encoder_layers],
    Dense(2*latent_dim),
])

In [14]:
decoder = tf.keras.Sequential([
    InputLayer(input_shape=(latent_dim,)),
    *[Dense(x) for x in decoder_layers],
    Dense(trb.shape[-1])
])

In [15]:
from VariationalAutoEncoder import VAE

vae = VAE(8, lambda *args: encoder, lambda *args: decoder, kl_importance=0)

In [16]:
import time

from IPython import display
epochs = 100

for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in train_dataset:
    vae.compute_apply_gradients(train_x)
  end_time = time.time()

  if epoch % 1 == 0:
    loss = tf.keras.metrics.Mean()
    for test_x in test_dataset:
      loss(vae.compute_loss(test_x))
    elbo = -loss.result()
    display.clear_output(wait=False)
    print('Epoch: {}, Test set ELBO: {}, '
          'time elapse for current epoch {}'.format(epoch,
                                                    elbo,
                                                    end_time - start_time))

Epoch: 100, Test set ELBO: -0.0063180080614984035, time elapse for current epoch 0.35124993324279785
